![POLITICO](https://rawgithub.com/The-Politico/src/master/images/logo/badge.png)

# POLITICO political district similarity maps

POLITICO district political similarity maps create lists of the most similar districts to any one district based on voting history.

The maps are created by calculating a weighted Euclidean distance between major party returns in each district.

---

## Data

Voting data used to determine similarity includes major party returns for:
- 2012, 2014 & 2016 U.S. House
- 2012 & 2016 U.S. president

The [data](https://www.politico.com/election-results/2018/race-ratings/data/historical/house.json) is sourced from MIT Election Lab and Daily Kos.

In [1]:
import requests

response = requests.get('https://www.politico.com/election-results/2018/race-ratings/data/historical/house.json')
data = response.json()
districts = [district for district in list(data.keys()) if 'LA' not in district and 'PA' not in district]
districts.sort()

---

## Processing

We use vote percent to standardize returns in each district.

In [2]:
def get_results(year, results):
    result = list(filter(lambda x: x['year'] == year, results))[0]
    dem = result.get('dem', {}).get('votePct', 0)
    gop = result.get('gop', {}).get('votePct', 0)
    return [dem, gop]

### Uncontested race discount

If a party is uncontested for a House seat during a cycle but has been contested in one or more other cycles, we discount the party's result in the uncontested race. We do this to limit the effect party dominance in any single uncontested race has because it may otherwise skew how competitive the district truly is.

The discount is arbitrary and calculated as half the return in the uncontested race minus the maximum return the party received in contested races during the period.

For example, if a Democratic candidate was uncontested in one cycle and received 95% of the vote but had only received a maximum of 55% of the vote in contested races, the uncontested result is discounted by (95 - 55) / 2 = 20%. The discounted return in the uncontested year would be **75%**.

If *all* races in the period are uncontested, we **do not** discount the uncontested return.



In [3]:
from statistics import mean

def flatten(list_of_lists):
    return [item for sublist in list_of_lists for item in sublist]

def discount_uncontested(results):
    uncontested = [result for result in results if 0 in result]
    contested = [result for result in results if 0 not in result]
    # If all races are uncontested, do not discount result
    if len(contested) == 0:
        return flatten(uncontested)
    
    discounted_uncontested = []

    for result in uncontested:
        # No Dem, GOP uncontested
        if result.index(0) == 0:
            avg_return = max([r[1] for r in contested])
            discount = (result[1] - avg_return) / 2
            discounted_uncontested.append([0, result[1] - discount])
        # No GOP, Dem uncontested
        else:
            avg_return = mean([result[0] for result in contested])
            discount = (result[0] - avg_return) / 2
            discounted_uncontested.append([result[0] - discount, 0])
    return flatten(discounted_uncontested) + flatten(contested)

In [4]:
district_results = {}

for district in districts:
    seat = data[district]['seat']
    pres = data[district]['president']
    
    # List of results [Dem, GOP]
    h2012 = get_results('2012', seat)
    h2014 = get_results('2014', seat)
    h2016 = get_results('2016', seat)
    p2012 = get_results('2012', pres)
    p2016 = get_results('2016', pres)
    
    house_results = discount_uncontested([h2012, h2014, h2016])
    
    district_results[district] = house_results + p2012 + p2016
    

### Weighted Euclidean distance


Each major party result represents a coordinate point along an axis of 0 to 1 (100% of the vote). Using those coordinates, we calculate the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between each district.

We weight results in calculating that distance, favoring recent results 2 to 1.

In [5]:
from scipy.spatial import distance

weights = {
    'h2012': 1, # House results
    'h2014': 1,
    'h2016': 2,
    'p2012': 1, # President results
    'p2016': 2,
}

def get_distance(district, comparitor):
    w = [
        #       DEM               GOP
        weights['h2012'], weights['h2012'],
        weights['h2014'], weights['h2014'],
        weights['h2016'], weights['h2016'],
        weights['p2012'], weights['p2012'],
        weights['p2016'], weights['p2016'],
    ]
    return distance.euclidean(district, comparitor, w)
    

In [6]:
district_distances = {}

for district, districtResults in district_results.items():
    district_distances[district] = []
    for comparitor, comparitorResults in district_results.items():
        if district == comparitor:
            continue
        district_distances[district].append({
            'district': comparitor,
            'distance': get_distance(districtResults, comparitorResults)
        })

For each district we calculate the **22** (~5%) most similar districts.

In [7]:
N = 22

In [8]:
similar_districts = {}
similar_districts_diagnostics = {}

for district, distances in district_distances.items():
    sorted_distances = list(sorted(distances, key=lambda k: k['distance']))
    similar_districts[district] = [k['district'] for k in sorted_distances[:N]]
    similar_districts_diagnostics[district] = sorted_distances[:N]

---

## Output

JSON

In [9]:
import json

with open('data/political-similarity.json', 'w') as file:
    json.dump(similar_districts, file)

CSV for diagnostics

In [10]:
import csv

with open('data/political-similarity-diagnostics.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(['district', 'min_similarity', 'max_similarity', 'similarity_range', 'most_similar'])

    for district in districts:
        min = similar_districts_diagnostics[district][0]['distance']
        max = similar_districts_diagnostics[district][-1]['distance']
        row = [district, min, max, max-min] + [k['district'] for k in similar_districts_diagnostics[district]]
        
        writer.writerow(row)
        
        print(row[0], row[4:14])

AK-00 ['WI-06', 'OK-05', 'MI-08', 'AR-02', 'WA-04', 'TX-25', 'WA-05', 'FL-03', 'TX-10', 'NC-06']
AL-01 ['GA-08', 'GA-10', 'GA-03', 'OK-01', 'TX-05', 'FL-04', 'TX-03', 'AZ-08', 'CA-23', 'GA-14']
AL-02 ['IN-09', 'FL-03', 'TX-27', 'FL-17', 'KS-04', 'TX-25', 'CA-01', 'NC-05', 'TX-31', 'SD-00']
AL-03 ['FL-11', 'IN-04', 'TN-03', 'MO-06', 'MS-01', 'KY-04', 'OH-04', 'WV-01', 'NC-03', 'FL-19']
AL-04 ['GA-14', 'TX-04', 'TX-11', 'TX-08', 'TX-19', 'TX-13', 'KY-05', 'TN-01', 'GA-09', 'NE-03']
AL-05 ['MO-04', 'MO-03', 'AL-03', 'MS-01', 'MO-06', 'IN-04', 'MD-01', 'FL-11', 'KY-04', 'SC-03']
AL-06 ['TX-01', 'KY-01', 'AZ-04', 'OK-03', 'FL-01', 'TN-07', 'OK-04', 'TN-02', 'TN-08', 'SC-03']
AL-07 ['GA-13', 'TX-29', 'AZ-07', 'MA-05', 'VA-03', 'CA-28', 'OR-03', 'CA-05', 'TX-30', 'SC-06']
AR-01 ['VA-09', 'CO-05', 'VA-06', 'SC-04', 'OH-07', 'AR-04', 'KY-02', 'TX-17', 'MS-03', 'TX-26']
AR-02 ['MI-06', 'MI-03', 'MI-08', 'WI-06', 'MI-07', 'NC-06', 'NC-02', 'KS-02', 'OK-05', 'TX-06']
AR-03 ['TX-19', 'TX-08', 'TN-0

In [11]:
from statistics import pvariance

response = requests.get('https://www.politico.com/election-results/2018/race-ratings/data/ratings.json')
ratings = {}
rating_codes = {}

for rating in response.json():
    ratings[rating['id']] = rating['latest_rating']['short_label']
    rating_codes[ratings[rating['id']]] = rating['latest_rating']['order']

    
with open('data/political-similarity-ratings.csv','w') as file:
    writer = csv.writer(file)
    writer.writerow(['district', 'rating', 'variance', 'most_similar_ratings'])
    
    for district in districts:
        row = [
            district,
            ratings[district],
            pvariance([rating_codes[ratings[k['district']]] for k in similar_districts_diagnostics[district]])
        ] + [ratings[k['district']] for k in similar_districts_diagnostics[district]]
        
        writer.writerow(row)
        
        print(row[0:2], row[3:8])

['AK-00', 'Likely-R'] ['Lean-R', 'Likely-R', 'Toss-Up', 'Lean-R', 'Solid-R']
['AL-01', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['AL-02', 'Solid-R'] ['Likely-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Likely-R']
['AL-03', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['AL-04', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['AL-05', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['AL-06', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['AL-07', 'Solid-D'] ['Solid-D', 'Solid-D', 'Solid-D', 'Solid-D', 'Solid-D']
['AR-01', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Likely-R']
['AR-02', 'Lean-R'] ['Likely-R', 'Solid-R', 'Toss-Up', 'Lean-R', 'Likely-R']
['AR-03', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['AR-04', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Likely-R']
['AZ-01', 'Lean-D'] ['Likely-D', 'Lean-D', 'Lean-R', 'Likely-D', 'Lean-R

['OH-10', 'Likely-R'] ['Likely-R', 'Likely-R', 'Lean-R', 'Likely-R', 'Lean-R']
['OH-11', 'Solid-D'] ['Solid-D', 'Solid-D', 'Solid-D', 'Solid-D', 'Solid-D']
['OH-12', 'Lean-R'] ['Likely-R', 'Likely-R', 'Likely-R', 'Likely-R', 'Solid-R']
['OH-13', 'Solid-D'] ['Likely-D', 'Solid-D', 'Solid-D', 'Solid-D', 'Solid-D']
['OH-14', 'Likely-R'] ['Solid-R', 'Likely-R', 'Lean-R', 'Toss-Up', 'Likely-R']
['OH-15', 'Likely-R'] ['Solid-R', 'Likely-R', 'Solid-R', 'Likely-R', 'Solid-R']
['OH-16', 'Solid-R'] ['Likely-R', 'Solid-R', 'Lean-R', 'Solid-R', 'Toss-Up']
['OK-01', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['OK-02', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['OK-03', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['OK-04', 'Solid-R'] ['Solid-R', 'Solid-R', 'Solid-R', 'Solid-R', 'Solid-R']
['OK-05', 'Likely-R'] ['Solid-R', 'Likely-R', 'Solid-R', 'Likely-R', 'Likely-R']
['OR-01', 'Solid-D'] ['Solid-D', 'Solid-D', 'Solid-D', 'Solid